In this notebook we look at how to use the gradient of the horsetail matching metric to speed up optimizations.

In [1]:
from horsetailmatching import UncertainParameter, HorsetailMatching
from horsetailmatching.demoproblems import TP1, TP2

First we will look at the purely probabilistic case and a simple test problem. We set up the uncertain parameters and create the horsetail matching object as usual. Note that in order to propagate gradients, we must use the kernel based method of evaluating the horsetail matching metric. 

In [2]:
u1 = UncertainParameter('uniform', lower_bound=-1, upper_bound=1)
u2 = UncertainParameter('uniform', lower_bound=-1, upper_bound=1)
input_uncertainties = [u1, u2]

Horsetail matching uses the same syntax for specifying a gradient as the scipy.minimize function, through the 'jacobian' argument. If jacobian is True, then horsetail matching expects the function provided to evaluate the qoi to also return the jacobian of the qoi (the gradient with respect to the design variables) as a second parameter. Alternatively jacobian is a fuction that similarly to the qoi function takes two inputs, the values of the design variables and uncertainties, and returns the gradient. The following code demonstrates these alternatives:

In [6]:
fun_qjac = lambda x, u: TP2(x, u, jac=True)
fun_q = lambda x, u: TP2(x, u, jac=False)
fun_jac = lambda x, u: TP2(x, u, jac=True)[1]

theHM = HorsetailMatching(fun_qjac, input_uncertainties, jacobian=True, method='kernel', 
                          n_samples_prob=2000, n_integration_points=1000, q_low=0, q_high=20)
theHM = HorsetailMatching(fun_q, input_uncertainties, jacobian=fun_jac, method='kernel', 
                          n_samples_prob=2000, n_integration_points=1000)
print(theHM.evalMetric([2,3]))

(11.711158705931167, array([-0.0401575 ,  0.04242819]))


Now we can use this in a gradient based optimizer:

In [8]:
from scipy.optimize import minimize

solution = minimize(theHM.evalMetric, x0=[1,1], method='CG', jac=True)
print(solution)

  status: 0
 success: True
    njev: 522
    nfev: 522
     fun: 9.96322479014987
       x: array([ -4.04850338, -12.37838541])
 message: 'Optimization terminated successfully.'
     jac: array([  9.01773800e-06,   7.55502293e-06])
